<a href="https://colab.research.google.com/github/IreneJeong/Project-Storage/blob/main/MLS_CW_Simple_CNN_%2B_TL_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Activation
from keras import losses
from keras.optimizers import Adam, Adagrad, SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import utils
import keras
from keras.layers import BatchNormalization

from tensorflow.keras.datasets import cifar10, cifar100

import numpy as np
import matplotlib.pyplot as plt
import os 

In [ ]:
import random
import tensorflow as tf
import os


def set_seed(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [ ]:
from tensorflow.keras.datasets import cifar10, cifar100

(cifar10_X_train, cifar10_y_train),(cifar10_X_test, cifar10_y_test)=cifar10.load_data()

print('input_X_train shape: ', cifar10_X_train.shape)
print(cifar10_X_train.shape[0], 'train samples')
print(cifar10_X_test.shape[0], 'test samples')

cifar10_N_CLASSES = len(np.unique(cifar10_y_train))
print(cifar10_N_CLASSES)

# output data one-hot encoding 
cifar10_y_train=utils.to_categorical(cifar10_y_train, cifar10_N_CLASSES)
cifar10_y_test=utils.to_categorical(cifar10_y_test, cifar10_N_CLASSES)

# To normalize the value in between 0 and 1 (there are 255 kinds)
cifar10_X_train=cifar10_X_train.astype('float32')
cifar10_X_test=cifar10_X_test.astype('float32')

cifar10_X_train /=255
cifar10_X_test /=255

input_X_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples
10


# Simple CNN and TL (ResNet50 and EB0)

## Simple CNN - CIFAR 10

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Define the CNN model
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Get the highest accuracy achieved during training
highest_accuracy = max(history.history["val_accuracy"])
print("Highest accuracy achieved:", highest_accuracy)


Epoch 1/10
1250/1250 [==============================] - 63s 50ms/step - loss: 1.5383 - accuracy: 0.4381 - val_loss: 1.2805 - val_accuracy: 0.5392
Epoch 2/10
1250/1250 [==============================] - 59s 47ms/step - loss: 1.1801 - accuracy: 0.5795 - val_loss: 1.1038 - val_accuracy: 0.6079
Epoch 3/10
1250/1250 [==============================] - 62s 49ms/step - loss: 1.0302 - accuracy: 0.6368 - val_loss: 1.0003 - val_accuracy: 0.6479
Epoch 4/10
1250/1250 [==============================] - 61s 48ms/step - loss: 0.9263 - accuracy: 0.6745 - val_loss: 0.9748 - val_accuracy: 0.6595
Epoch 5/10
1250/1250 [==============================] - 61s 49ms/step - loss: 0.8488 - accuracy: 0.7041 - val_loss: 0.9249 - val_accuracy: 0.6742
Epoch 6/10
1250/1250 [==============================] - 61s 49ms/step - loss: 0.7776 - accuracy: 0.7270 - val_loss: 0.9045 - val_accuracy: 0.6879
Epoch 7/10
1250/1250 [==============================] - 63s 50ms/step - loss: 0.7257 - accuracy: 0.7469 - val_loss: 0.8853 -

## Simple CNN - CIFAR100

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar100

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Define the CNN model
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(100),
    ]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train the model
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

# Get the highest accuracy achieved during training
highest_accuracy = max(history.history["val_accuracy"])
print("Highest accuracy achieved:", highest_accuracy)


169001437/169001437 [==============================] - 2s 0us/step
Epoch 1/50
1250/1250 [==============================] - 63s 49ms/step - loss: 4.0557 - accuracy: 0.0777 - val_loss: 3.7026 - val_accuracy: 0.1237
Epoch 2/50
1250/1250 [==============================] - 61s 49ms/step - loss: 3.4322 - accuracy: 0.1779 - val_loss: 3.2796 - val_accuracy: 0.2073
Epoch 3/50
1250/1250 [==============================] - 60s 48ms/step - loss: 3.1234 - accuracy: 0.2365 - val_loss: 3.1071 - val_accuracy: 0.2418
Epoch 4/50
1250/1250 [==============================] - 61s 49ms/step - loss: 2.9164 - accuracy: 0.2764 - val_loss: 2.9702 - val_accuracy: 0.2664
Epoch 5/50
1250/1250 [==============================] - 59s 47ms/step - loss: 2.7709 - accuracy: 0.3049 - val_loss: 2.9006 - val_accuracy: 0.2777
Epoch 6/50
1250/1250 [==============================] - 62s 49ms/step - loss: 2.6489 - accuracy: 0.3294 - val_loss: 2.8944 - val_accuracy: 0.2863
Epoch 7/50
1250/1250 [==============================] - 6

## TL ResNet50 - CIFAR100

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new trainable layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(100, activation="softmax")(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)


94765736/94765736 [==============================] - 4s 0us/step


Epoch 1/10
313/313 [==============================] - 228s 714ms/step - loss: 4.5634 - accuracy: 0.0244 - val_loss: 4.4492 - val_accuracy: 0.0336
Epoch 2/10
313/313 [==============================] - 214s 685ms/step - loss: 4.3519 - accuracy: 0.0483 - val_loss: 4.2763 - val_accuracy: 0.0587
Epoch 3/10
313/313 [==============================] - 218s 695ms/step - loss: 4.2281 - accuracy: 0.0683 - val_loss: 4.2339 - val_accuracy: 0.0628
Epoch 4/10
313/313 [==============================] - 216s 691ms/step - loss: 4.1559 - accuracy: 0.0781 - val_loss: 4.1652 - val_accuracy: 0.0728
Epoch 5/10
313/313 [==============================] - 215s 688ms/step - loss: 4.1019 - accuracy: 0.0844 - val_loss: 4.0984 - val_accuracy: 0.0865
Epoch 6/10
313/313 [==============================] - 214s 686ms/step - loss: 4.0597 - accuracy: 0.0920 - val_loss: 4.0510 - val_accuracy: 0.0931
Epoch 7/10
313/313 [==============================] - 216s 689ms/step - loss: 4.0234 - accuracy: 0.0968 - val_loss: 4.0440 -

## TL - EB0 Cifar100

In [3]:
!pip install efficientnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from efficientnet.tfkeras import EfficientNetB0

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Define the input shape
input_shape = x_train.shape[1:]

# Use data augmentation to reduce overfitting
data_augmentation = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)

# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)

# Add new layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(100, activation="softmax")(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(lr=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train the model with transfer learning
history = model.fit(
    data_augmentation.flow(x_train, y_train, batch_size=32),
    steps_per_epoch=len(x_train) // 32,
    epochs=50,
    validation_data=(x_test, y_test),
    callbacks=[EarlyStopping(patience=5)],
)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)


16804768/16804768 [==============================] - 0s 0us/step


Epoch 1/50
1562/1562 [==============================] - 194s 114ms/step - loss: 3.7608 - accuracy: 0.0150 - val_loss: 3.3131 - val_accuracy: 0.0108
Epoch 2/50
1562/1562 [==============================] - 158s 101ms/step - loss: 3.3659 - accuracy: 0.0133 - val_loss: 3.1443 - val_accuracy: 0.0184
Epoch 3/50
1562/1562 [==============================] - 154s 99ms/step - loss: 3.2495 - accuracy: 0.0129 - val_loss: 3.0856 - val_accuracy: 0.0140
Epoch 4/50
1562/1562 [==============================] - 155s 99ms/step - loss: 3.1729 - accuracy: 0.0123 - val_loss: 3.0399 - val_accuracy: 0.0053
Epoch 5/50
1562/1562 [==============================] - 156s 100ms/step - loss: 3.1226 - accuracy: 0.0117 - val_loss: 2.9608 - val_accuracy: 0.0125
Epoch 6/50
1562/1562 [==============================] - 156s 100ms/step - loss: 3.0803 - accuracy: 0.0121 - val_loss: 2.9696 - val_accuracy: 0.0071
Epoch 7/50
1562/1562 [==============================] - 152s 97ms/step - loss: 3.0547 - accuracy: 0.0120 - val_los